In [7]:
import jupyter_black

jupyter_black.load()  # Python linter

# Automatically provide the LLM with the calculator tool

In [8]:
%cat ../baml_src/02_use_tool.baml

class ToolCall {
    @@dynamic
}

function UseTool(goal: string, output_format_prefix: string) -> ToolCall {
    client Default
    prompt #"
        Solve: {{ goal }}

        {{ ctx.output_format(prefix=output_format_prefix, or_splitter=" OR ")}}
    "#
}


In [10]:
from baml_agents import McpServers, view_prompt
from pydantic_ai.mcp import MCPServerStdio

from baml_client.type_builder import TypeBuilder
from baml_client.async_client import b

server_with_calculator_tool = McpServers(
    [
        MCPServerStdio(
            command="python",
            args=["-m", "mcp-server-calculator"],
        )
    ]
)

In [ ]:
async with server_with_calculator_tool as server:
    # Add MCP tools to the BAML structured output schema
    tb, tool_runner, p = await server.build_tool_types(
        tb := TypeBuilder(),
        output_class=tb.ToolCall,
        tools=await server.list_tools(),
    )

    # View prompt
    goal = "Multiply all numbers between 10 and 15"
    request = await b.request.UseTool(goal, p, baml_options={"tb": tb})
    print(view_prompt(request))

    # LLM chooses the tools
    result = await b.UseTool(goal, p, baml_options={"tb": tb})

    # Run the tools
    tool_results = await tool_runner.run(result)

In [4]:
result

ToolCall(intents=[{'intent': 'calculate', 'expression': '11 * 12 * 13 * 14'}])

In [5]:
tool_results

[CallToolResult(meta=None, content=[TextContent(type='text', text='24024', annotations=None)], isError=False)]